In [ ]:
import pathlib
import pandas as pd

datadir = pathlib.Path.cwd().parent / "data"

pd_hotel       = pd.read_parquet(datadir / "hotel.parquet")
pd_customer    = pd.read_parquet(datadir / "customer.parquet")
pd_reservation = pd.read_parquet(datadir / "reservation.parquet")
pd_campaign    = pd.read_parquet(datadir / "campaign.parquet")

# 7章 結合


## 7-1 1対1または多対1の関係のテーブルの結合
### Q: ビジネスホテルかつ宿泊人数が1名の予約履歴の抽出


#### Not Awesome

In [ ]:
(
    pd_reservation
    #（1）pd_reservationとpd_hotelをhotel_id列をキーとして内部結合
    .merge(pd_hotel, on="hotel_id", how="inner")
    #（2）結合したデータからビジネスホテルかつ宿泊1名の行を抽出
    .query("hotel_type == 'ビジネスホテル' & people_num == 1")
)

#### Awesome

In [ ]:
(
    pd_reservation
    #（1）people_numが1人のデータのみ抽出
    .query("people_num == 1")
    #（3）ビジネスホテルのみのマスタを内部結合
    .merge(
        #（2）pd_hotelからビジネスホテルの行のみ抽出
        pd_hotel.query("hotel_type == 'ビジネスホテル'")["hotel_id"],
        on="hotel_id", how="inner"
    )
)

## 7-2 1対多の関係のテーブルの結合
### Q: ホテルマスタにホテルの売上と予約数を付与


#### Not Awesome

In [ ]:
(
    pd_hotel
    #（1）hotel_id列を結合キーとしてpd_reservationを左外部結合
    .merge(pd_reservation, on="hotel_id", how="left")
    #（2）未キャンセルかつcheckout_dateの年が2019の行のみ抽出
    .query("status != 'canceled' and checkout_date.dt.year == 2019")
    #（3）集計結果以外に結果に残す列を全て結合キーに指定してgroup by
    .groupby(["hotel_id","hotel_name","hotel_type","address_prefecture",
              "address_city","address_town","address_zipcode","unit_price",
              "user_rating"]).total_price.agg(["sum", "size"])
)

#### Awesome

In [ ]:
(
    pd_hotel
    #（2）hotel_id列を結合キーとして（1）の集計結果を左外部結合
    .merge(
        #（1）pd_reservationから未キャンセルかつcheckout_dateの年が2019の行のみ抽出し、
        #    hotel_idを集約キーとしてgroup by集計
        pd_reservation
        .query("status != 'canceled' and checkout_date.dt.year == 2019")
        .groupby("hotel_id").total_price.agg(["sum", "size"]),
        on="hotel_id", how="left"
    )
    #（3）結合されなかった行の数値を0埋め
    .fillna({"sum": 0, "size": 0})
)

## 7-3 多対多の関係のテーブルの結合
### Q: 顧客マスタに対して、顧客のホテル種別ごとの予約数を付与


#### Awesome

In [ ]:
import numpy as np

(
    pd_customer
    #（4）pd_customerに（1）〜（3）の集計結果を左外部結合
    .merge(
        pd_reservation
        #（1）pd_reservationにpd_hotelを左外部結合
        .merge(pd_hotel[["hotel_id", "hotel_type"]], on="hotel_id", how="left")
        #（2）ホテル種別ごとのデータ数カウント用の列を作成
        .assign(
            ryokan_cnt=lambda df: np.where(df.hotel_type == "旅館", 1, 0),
            resort_hotel_cnt=lambda df:
                np.where(df.hotel_type == "リゾートホテル", 1, 0),
            business_hotel_cnt=lambda df:
                np.where(df.hotel_type == "ビジネスホテル", 1, 0),
            minsyuku_cnt=lambda df: np.where(df.hotel_type == "民宿", 1, 0)
        )
        #（3）customer_idごとにホテル種別ごとのデータ数をカウント
        .groupby("customer_id")[["ryokan_cnt", "resort_hotel_cnt",
                                "business_hotel_cnt", "minsyuku_cnt"]].sum(),
        on="customer_id", how="left"
    )
    #（5）結合されなかった行の数値を0埋め
    .fillna({"ryokan_cnt": 0, "resort_hotel_cnt": 0,
            "business_hotel_cnt": 0, "minsyuku_cnt": 0})
)

## 7-4 すべての結合の組み合わせの生成
### Q: 顧客ごとの月別の売上を計算（売上のない月も出力）


#### Awesome

In [ ]:
(
    pd_customer[["customer_id"]]
    #（2）pd_customerに対して月の時系列をクロス結合し、customer_idごとに全ての月の行を生成
    .merge(
        #（1）1ヶ月間隔の時系列を生成
        pd.period_range("2019-01", "2019-12", freq="M").to_series(name="month"),
        how="cross"
    )
    #（6）（2）のデータに対して（3）〜（5）の集計結果を左外部結合
    .merge(
        pd_reservation
        #（3）未キャンセルデータを抽出
        .query("status != 'canceled'")
        #（4）checkout_dateを月周期のPeriod型に変換
        .assign(month=lambda df: df.checkout_date.dt.to_period("M"))
        #（5）customer_idとmonthごとにtotal_priceの総和を計算
        .groupby(["customer_id", "month"]).total_price.sum(),
        on=["customer_id", "month"], how="left"
    )
    #（7）結合されなかった行の数値を0埋め
    .fillna({"total_price": 0})
)

## 7-5 不等式条件での結合
### Q: 予約履歴データにキャンペーン情報を付与


#### Not Awesome

In [ ]:
(
    pd_reservation
    #（1）pd_reservationにpd_campaignをクロス結合
    .merge(pd_campaign, how="cross")
    #（2）pd_reservationのreserved_atがpd_campaignのstarts_atとends_atの間にある行のみ抽出
    .query("starts_at <= reserved_at <= ends_at")
    #（3）不要な列を削除
    .drop(columns=["starts_at", "ends_at"])
)

#### Awesome

In [ ]:
(
    pd_reservation
    #（1）reserved_atを日付単位に丸めたreserve_date列を作成
    .assign(reserve_date=lambda df: df.reserved_at.dt.normalize())
    #（4）reserve_dateを結合キーとしてキャンペーンマスタと結合
    .merge(
        pd_campaign
        #（2）starts_atとends_atの間の日付単位の時系列を作成
        .assign(reserve_date=lambda df:
            df.apply(lambda r: pd.date_range(r.starts_at, r.ends_at), axis=1))
        [["campaign_name", "reserve_date"]]
        #（3）日付の時系列を複数行に展開
        .explode("reserve_date"),
        on="reserve_date", how="left"
    )
    #（5）不要な列を削除
    .drop(columns="reserve_date")
)